<a href="https://www.kaggle.com/code/gautamnaik1994/netflix-case-study?scriptVersionId=151138883" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <a id='toc1_'></a>[Netflix Case Study](#toc0_)

By Gautam Naik (gautamnaik1994@gmail.com)

Google Collab Link: https://colab.research.google.com/github/gautamnaik1994/NetflixDataAnalysisCaseStudy/blob/main/CaseStudy.ipynb  
Github Repo Link: https://github.com/gautamnaik1994/NetflixDataAnalysisCaseStudy  
Github Pages Link: https://gautamnaik1994.github.io/NetflixDataAnalysisCaseStudy/

**Business Problem**
- Help Netflix in deciding which type of shows/movies to produce 
- How to grow the business in different countries

**Metric**
- Since there is not data about views count, user star rating we are going to use the count of content added to Netflix as the metric.
- We will also use the count of cast, director, rating as measure of popularity.

**Table of contents**<a id='toc0_'></a>    
- [Netflix Case Study](#toc1_)    
  - [Data Cleaning and Splitting](#toc1_1_)    
    - [Separating nested data](#toc1_1_1_)    
    - [Adding date related columns](#toc1_1_2_)    
    - [Exporting data to separate files](#toc1_1_3_)    
  - [EDA and Insights](#toc1_2_)    
    - [Country Analysis](#toc1_2_1_)    
    - [Cast Analysis](#toc1_2_2_)    
    - [Genre Analysis](#toc1_2_3_)    
      - [Movie Genre Analysis](#toc1_2_3_1_)    
      - [TV Show Genre Analysis](#toc1_2_3_2_)    
    - [Director Analysis](#toc1_2_4_)    
    - [Release Timeline Analysis](#toc1_2_5_)    
      - [Analysis of all time data](#toc1_2_5_1_)    
      - [Analysis of latest data](#toc1_2_5_2_)    
    - [Movie and TV Show Distribution Analysis](#toc1_2_6_)    
      - [TV Show popularity analysis](#toc1_2_6_1_)    
    - [Rating Analyisis](#toc1_2_7_)    
    - [Duration Anaylsis](#toc1_2_8_)    
      - [TV Show Analysis](#toc1_2_8_1_)    
      - [Movie Analysis](#toc1_2_8_2_)    
  - [Recommendations](#toc1_3_)    
    - [General Recommendations](#toc1_3_1_)    
    - [Content Recommendations](#toc1_3_2_)    
      - [Top genres for each country](#toc1_3_2_1_)    
      - [Top rated content for each country](#toc1_3_2_2_)    
      - [Top cast in each country](#toc1_3_2_3_)    
      - [Top directors in each country](#toc1_3_2_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
!pip install duckdb

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import duckdb
import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
sns.set_style('darkgrid')
pd.reset_option('display.max_rows')

## <a id='toc1_1_'></a>[Data Cleaning and Splitting](#toc0_)

In [ ]:
df=pd.read_csv('../input/netflix-data/netflix.csv')
df.sample(10)

In [ ]:
df = df.drop(["description","title"], axis=1)
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df["type"].value_counts()
df["rating"].value_counts()

In [ ]:
df["type"]=df["type"].astype("category")
mask=df["rating"].isin(["74 min","84 min","66 min"])
df.loc[mask, "duration"] = df.loc[mask, "rating"]
df.loc[mask, "rating"] = df["rating"].mode().iloc[0]
df["rating"]=df["rating"].fillna(df["rating"].mode().iloc[0])
df["date_added"] = pd.to_datetime(df["date_added"] , format='%B %d, %Y', errors="coerce")
mask=df["date_added"].isna()
df.loc[mask, "date_added"] = df.loc[mask, "release_year"].apply(lambda x: max(pd.to_datetime(x + 1, format="%Y"), pd.Timestamp(datetime.date(2006, 1, 1))).date() )

In [ ]:
# df.set_index('show_id')['cast'].str.split(', ', expand=True).stack().reset_index(name='cast').drop('level_1', axis=1)

### <a id='toc1_1_1_'></a>[Separating nested data](#toc0_)

In [ ]:
cast=df['cast'].apply(lambda x: str(x).split(', ')).tolist()
cast_df=pd.DataFrame(cast,index=df['show_id'])
cast_df=cast_df.stack().reset_index(name='cast').drop('level_1', axis=1).set_index('show_id')
cast_df.replace("nan", float('nan'), inplace=True)
# mask = cast_df[cast_df['cast'] == ''].index
# cast_df.drop(mask, inplace=True)
# cast_df

director=df['director'].apply(lambda x: str(x).split(', ')).tolist()
director_df=pd.DataFrame(director,index=df['show_id'])
director_df=director_df.stack().reset_index(name='director').drop('level_1', axis=1).set_index('show_id')
director_df.replace("nan", float('nan'), inplace=True)
# director_df

country=df['country'].apply(lambda x: str(x).split(', ')).tolist()
country_df=pd.DataFrame(country,index=df['show_id'])
country_df=country_df.stack().reset_index(name='country').drop('level_1', axis=1).set_index('show_id')
country_df.replace("nan", float('nan'), inplace=True)
mask = country_df[country_df['country'] == ''].index
country_df.drop(mask, inplace=True)
# country_df.replace(" ", float('nan'), inplace=True)
# country_df

listed=df['listed_in'].apply(lambda x: str(x).split(', ')).tolist()
listed_df=pd.DataFrame(listed,index=df['show_id'])
listed_df=listed_df.stack().reset_index(name='listed_in').drop('level_1', axis=1).set_index('show_id')
listed_df.replace("nan", float('nan'), inplace=True)
# listed_df
df.drop(["cast","country","director","listed_in"], axis=1, inplace=True)

### <a id='toc1_1_2_'></a>[Adding date related columns](#toc0_)

In [ ]:
df["duration"]=df["duration"].apply(lambda x: x.split(" ")[0])
df["date_added_year_month"] = df["date_added"].dt.strftime('%Y-%m')
df["date_added_year"]=df["date_added"].dt.year
df["date_added_month"]=df["date_added"].dt.month
df["date_added_month_name"]=df["date_added"].dt.month_name()
df["dat_added_period"]=pd.cut(df["date_added_year"], bins=[0,2005, 2010, 2015,2022 ], labels=["2005","2006-2010","2011-2015","2016-2022"])

movies_df=df.loc[df["type"]=="Movie"]
tv_shows_df=df.loc[df["type"]=="TV Show"]
movies_df.head()
tv_shows_df.head()

In [ ]:
country_df.value_counts()
cast_df.value_counts()
director_df.value_counts()

In [ ]:
country_df["country"].mode().iloc[0]

In [ ]:
director_df.fillna("Unknown", inplace=True)
cast_df.fillna("Unknown", inplace=True)
country_df=country_df.fillna('Unknown')

In [ ]:
country_df.reset_index(inplace=True)
cast_df.reset_index(inplace=True)
listed_df.reset_index(inplace=True)
director_df.reset_index(inplace=True)

In [ ]:
country_df.isna().sum()
director_df.isna().sum()
listed_df.isna().sum()
tv_shows_df.isna().sum()
movies_df.isna().sum()
df.isna().sum()
cast_df.isna().sum()

### <a id='toc1_1_3_'></a>[Exporting data to separate files](#toc0_)

In [ ]:
# country_df.to_csv("country.csv", index=False)
# director_df.to_csv("director.csv", index=False)
# cast_df.to_csv("cast.csv", index=False)
# listed_df.to_csv("listed.csv", index=False)
# df.to_csv("data.csv", index=False)
# movies_df.to_csv("movies.csv", index=False)
# tv_shows_df.to_csv("tv_shows.csv", index=False)

## <a id='toc1_2_'></a>[EDA and Insights](#toc0_)

In [ ]:
country_df=pd.read_csv("../input/netflix-data/country.csv")
cast_df=pd.read_csv("../input/netflix-data/cast.csv")
listed_df=pd.read_csv("../input/netflix-data/listed.csv")
movies_df=pd.read_csv("../input/netflix-data/movies.csv", parse_dates=["date_added"])
tv_shows_df=pd.read_csv("../input/netflix-data/tv_shows.csv",parse_dates=["date_added"])
director_df=pd.read_csv("../input/netflix-data/director.csv")
df=pd.read_csv("../input/netflix-data/data.csv",parse_dates=["date_added"])
# df["type"]=df["type"].astype("category")
# movies_df["type"]=movies_df["type"].astype("category")
# tv_shows_df["type"]=tv_shows_df["type"].astype("category")

In [ ]:
df.describe( include="all" )

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- There are 8807 unique shows and movies availble in the dataset.

In [ ]:
sns.pairplot(df, hue="type")

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- No particular paatern is observed in above plot

In [ ]:
sns.heatmap(tv_shows_df[["date_added_year","date_added_month","duration","release_year"]].corr(), annot=True, cmap="viridis");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- There is no strong correlation between the columns of the tv show data.

In [ ]:
sns.heatmap(movies_df[["date_added_year","date_added_month","duration","release_year"]].corr(), annot=True, cmap="viridis");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- There is no strong correlation between the columns of the movie data.

### <a id='toc1_2_1_'></a>[Country Analysis](#toc0_)

In [ ]:
country_df.describe()

In [ ]:
cdf=country_df["country"].value_counts()[:25]
cdf

In [ ]:

plt.figure(figsize=(18,5))
plt.bar(cdf.index,cdf)
plt.xticks(rotation=45)
plt.ylabel("Content Count")
plt.xlabel("Country")
plt.title("Top Content Producers");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- From above graph we can see that USA is a top content producer.
- Countries like India, UK, Canada, France, Japan, Spain, South Korea and Germany have lot of scope for improvement.
- Countries after Germany have a very high scope for improvement.

In [ ]:
plt.figure(figsize=(10,10))
plt.pie(cdf, labels=cdf.index, autopct= '%1.1f%%')
plt.title("Coutent Distribution for each country");


<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- 37.4% content is produced by USA and 10.6% is produced by India, indicating the popularity of the content available on Netflix
- This is because Hollywood and Bollywood are biggest film industry in the world.

In [ ]:
improvement_countries= country_df["country"].value_counts()[1:25].drop(index="Unknown").index.to_list()
improvement_countries
# non_top_3_countrries

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- We will focus on above countries the most as there is a higher chance of growth if we invest in producing content for them

### <a id='toc1_2_2_'></a>[Cast Analysis](#toc0_)

In [ ]:
cast_df.describe()

In [ ]:
merge_df=df.merge(cast_df,on='show_id',how='inner')
merge_df.head()

In [ ]:
cdf = cast_df["cast"].value_counts()[:11].reset_index()
cdf

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Above table shows the list of top cast present in movies and tv shows
- There is lot of missing values in this data, which have been replaced by "Unknown"
- Anupam Kher appears to be in maximum number of content present on Netflix

In [ ]:
cdf = cdf.iloc[1:11]
mdf=merge_df.loc[merge_df["type"]=="Movie"]["cast"].value_counts()[1:11].reset_index()
tdf=merge_df.loc[merge_df["type"]=="TV Show"]["cast"].value_counts()[1:11].reset_index()
print("Overall Top Cast")
cdf
print("Movie Top Cast")
mdf
print("TV Show Top Cast")
tdf

In [ ]:
fig = plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(2, 2, width_ratios=[1, 1], hspace=0.5)


ax1 = plt.subplot(gs[0, :])
sns.barplot(data=cdf, x="cast", y="count", ax=ax1)
# ax1.tick_params(axis='x', labelrotation=45)
ax1.set_title("Top Cast in both TV Shows and Movies")

ax2 = plt.subplot(gs[1, 0])
sns.barplot(data=tdf, x="cast", y="count", ax=ax2)
ax2.tick_params(axis='x', labelrotation=45)
ax2.set_title("Top Cast in TV Shows")


ax3 = plt.subplot(gs[1, 1])
sns.barplot(data=mdf, x="cast", y="count", ax=ax3)
ax3.set_title("Top Cast in Movies")
ax3.tick_params(axis='x', labelrotation=45);


<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Above visuals shows the list of top cast present in movies and tv shows
- There appears to be lot of Indian actors in the movies section
- Takahiro Sakurai is the most popular actor in the TV show section
- Anupam Kher is the most popular actor in the movies section and overall section


In [ ]:
cast_count_df=duckdb.sql(""" 
           with cte as (
            select distinct show_id, type,country, cast_df.cast from df 
            join cast_df using(show_id) 
            join country_df using(show_id) 
            where country != 'Unknown' and cast_df.cast != 'Unknown' 
           )
           select country, sum(case when type ='TV Show' then 1 else 0 end) "tv cast count", 
           sum(case when type ='Movie' then 1 else 0 end) "movie cast count" from cte 
           group by country order by "tv cast count" desc
""").df()
cast_count_df.head(20)

In [ ]:
temp_df=cast_count_df.head(20).melt(id_vars=['country'],var_name="type", value_name='cast count')
plt.figure(figsize=(20,5))
lp=sns.barplot(data=temp_df, x="country", y="cast count", hue="type")
plt.xticks(rotation=45)
plt.title("Cast Count by Country");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- We can see that Japan, South Korea and Taiwan have more TV casts than movies indicating that users are more likely to watch TV shows


### <a id='toc1_2_3_'></a>[Genre Analysis](#toc0_)

In [ ]:
listed_df.describe()

In [ ]:
ldf=listed_df["listed_in"].value_counts()[:10].reset_index()
ldf

In [ ]:
merge_df=df.merge(listed_df,on='show_id',how='inner')
merge_df.head()

In [ ]:
mdf=merge_df.loc[merge_df["type"]=="Movie"]["listed_in"].value_counts()[:10].reset_index()
tdf=merge_df.loc[merge_df["type"]=="TV Show"]["listed_in"].value_counts()[:10].reset_index()
ldf
mdf
tdf

In [ ]:
fig = plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(2, 2, width_ratios=[1, 1], hspace=0.5)


ax1 = plt.subplot(gs[0, :])
sns.barplot(data=ldf, x="listed_in", y="count", ax=ax1)
# ax1.tick_params(axis='x', labelrotation=45)
ax1.set_title("Top Genre in both TV Shows and Movies")

ax2 = plt.subplot(gs[1, 0])
sns.barplot(data=tdf, x="listed_in", y="count", ax=ax2)
ax2.tick_params(axis='x', labelrotation=45)
ax2.set_title("Top Genre in TV Shows")


ax3 = plt.subplot(gs[1, 1])
sns.barplot(data=mdf, x="listed_in", y="count", ax=ax3)
ax3.set_title("Top Genre in Movies")
ax3.tick_params(axis='x', labelrotation=45);

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Above graphs shows the list of top genres present in movies and tv show category
- It appears that users like watching international movies/tv shows, dramas and comedies the most
- This indicates there is high demand for International content in both TV and movies sectiion


#### <a id='toc1_2_3_1_'></a>[Movie Genre Analysis](#toc0_)

In [ ]:
merge_df=movies_df.merge(listed_df, on='show_id', how='inner')
merge_df_trunc = merge_df.loc[merge_df['date_added_year'] >= 2014]
plt.figure(figsize=(15, 5))
sns.boxplot(data=merge_df_trunc, x="listed_in", y="date_added_year")
plt.title("Boxplot of date_added_year by listed_in for movies after 2014")
plt.xticks(rotation=90);

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Majority of the content has added during 2018 - 2020

In [ ]:
temp_df=merge_df.groupby(["date_added_year","listed_in"])["show_id"].count().reset_index()
temp_df=temp_df.loc[temp_df["show_id"]>50]
plt.figure(figsize=(20,5))
sns.lineplot(data=temp_df, y="show_id", x="date_added_year", hue="listed_in", palette="tab10" )
plt.xlim((2014,2022));

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- International movies, Dramas and Comedies have been added the most over last few years indicating popularity of those genre in movie section

#### <a id='toc1_2_3_2_'></a>[TV Show Genre Analysis](#toc0_)

In [ ]:
merge_df=tv_shows_df.merge(listed_df, on='show_id', how='inner')
merge_df_trunc = merge_df.loc[merge_df['date_added_year'] >= 2014]
plt.figure(figsize=(15, 5))
sns.boxplot(data=merge_df_trunc, x="listed_in", y="date_added_year")
plt.title('Boxplot of Date Added Year for TV Shows after 2014')
plt.xticks(rotation=90);

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Majority of the content has added during 2018 - 2020 indicating Netflix popularity.

In [ ]:
temp_df=merge_df.groupby(["date_added_year","listed_in"])["show_id"].count().reset_index()
temp_df=temp_df.loc[temp_df["show_id"]>50]
plt.figure(figsize=(20,5))
sns.lineplot(data=temp_df, y="show_id", x="date_added_year", hue="listed_in", palette="tab10" )
plt.xlim((2014,2022));

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- International TV shows have been added the most over last few years indicating popularity of this genre in TV show section

### <a id='toc1_2_4_'></a>[Director Analysis](#toc0_)

In [ ]:
director_df.describe()

In [ ]:
director_df.value_counts()

In [ ]:
director_df["director"].nunique()

In [ ]:
director_df.drop_duplicates()

In [ ]:
merge_df=df.merge(director_df,on='show_id',how='inner')
merge_df.head()

In [ ]:
ddf = director_df["director"].value_counts()[:11].reset_index()
ddf

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Above table shows the list of top directors present in movies and tv shows
- There is lot of missing values in this data, which have been replaced by "Unknown"

In [ ]:
ddf = ddf.iloc[1:11]
mdf=merge_df.loc[merge_df["type"]=="Movie"]["director"].value_counts()[1:11].reset_index()
tdf=merge_df.loc[merge_df["type"]=="TV Show"]["director"].value_counts()[1:11].reset_index()
ddf
mdf
tdf

In [ ]:
fig = plt.figure(figsize=(20, 10))
gs = gridspec.GridSpec(2, 2, width_ratios=[1, 1], hspace=0.5)


ax1 = plt.subplot(gs[0, :])
sns.barplot(data=ddf, x="director", y="count", ax=ax1)
ax1.set_title("Top director in both TV Shows and Movies")

ax2 = plt.subplot(gs[1, 0])
sns.barplot(data=tdf, x="director", y="count", ax=ax2)
ax2.tick_params(axis='x', labelrotation=45)
ax2.set_title("Top director in TV Shows")


ax3 = plt.subplot(gs[1, 1])
sns.barplot(data=mdf, x="director", y="count", ax=ax3)
ax3.set_title("Top director in Movies")
ax3.tick_params(axis='x', labelrotation=45);

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Above graphs shows the list of top directors present in movies and tv shows

### <a id='toc1_2_5_'></a>[Release Timeline Analysis](#toc0_)

#### <a id='toc1_2_5_1_'></a>[Analysis of all time data](#toc0_)

In [ ]:
stream_timeline=df.groupby(["date_added_year_month","type"])["show_id"].count().reset_index().set_index("date_added_year_month")
stream_timeline.head()

In [ ]:
plt.figure(figsize=(20,5))
# fig, ax = plt.subplots(figsize=(20, 5))
l=sns.lineplot(data=stream_timeline, x="date_added_year_month", y="show_id", hue="type")
l.set(xlabel="Year", ylabel="Count");
l.set_title("Content Upload Timeline");
l.set_xticks(l.get_xticks()[::5]);
# plt.gca().xaxis.set_major_locator(plt.MultipleLocator(10))
# plt.gca().xaxis.set_minor_locator(plt.MultipleLocator(1))

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- From above graph, we can see that majority of content got added after 2015.
- Over the years, the amount of tv shows added were comparatively less than movies.
- This shows that movies are more popular than tv shows.

In [ ]:
monthly_release=df.groupby(["date_added_month","date_added_month_name","type"])["show_id"].count().reset_index().set_index("date_added_month")
monthly_release.head()

In [ ]:
plt.figure(figsize=(20,5))

lp=sns.barplot(data=monthly_release, x="date_added_month_name", y="show_id", hue="type")
lp.set(xlabel="Month", ylabel="Count");
lp.set_title("All Time Monthly Releases");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- From above plot we can see that January, April, July and December months has highest number of content added.
- This seems to be ideal time to release content

In [ ]:
plt.figure(figsize=(10,5))
(movies_df["date_added_year"] - movies_df["release_year"]).hist(bins=50)
plt.title("Histogram of realease date and added date for movies")
plt.xlabel("Difference in years")
plt.ylabel("Count");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Majority of the movies are added under 1 year after the release date.
- This shows that people are eager to watch the movie after its theatrical release date.
- Netflix should produce more original content

#### <a id='toc1_2_5_2_'></a>[Analysis of latest data](#toc0_)

In [ ]:
latest_data=df.loc[df["dat_added_period"].isin(["2016-2022"])]

In [ ]:
monthly_release_latest=latest_data.groupby(["date_added_month","date_added_month_name","type"])["show_id"].count().reset_index().set_index("date_added_month")
monthly_release.head()

In [ ]:
plt.figure(figsize=(20,5))

lp=sns.barplot(data=monthly_release_latest, x="date_added_month_name", y="show_id", hue="type")
lp.set(xlabel="Month", ylabel="Count");
lp.set_title("Monthly Releases for period 2016-2021");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- From above plot we can see that January, April, July and December months has highest number of content added even for recent data.
- This seems to be ideal time to release content

In [ ]:
plt.figure(figsize=(20,5))
df["release_year"].value_counts().plot(kind="bar");
plt.ylabel("Count");
plt.title("Movie realease year distribution");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- From above graph, we can see that majority of content that is added was realeased after 2014
- This shows that more and more content producers are using Netflix to distribute their content.

### <a id='toc1_2_6_'></a>[Movie and TV Show Distribution Analysis](#toc0_)

In [ ]:
pdf=df["type"].value_counts()
pdf

In [ ]:
sns.countplot(data=df, x="type");

In [ ]:
plt.pie(pdf, labels=pdf.index, autopct='%.0f%%')
plt.title("Movie and TV show percentages");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- From above plot we can say that movies are more popular than TV shows
- This shows that users are more interested in stories that get over in under 2hrs.

#### <a id='toc1_2_6_1_'></a>[TV Show popularity analysis](#toc0_)

In [ ]:
tmdf=df.merge(country_df, on="show_id").groupby(["country","type"])["show_id"].count().reset_index().rename(columns={"show_id":"count"}).sort_values(by="count", ascending=False)
tmdf

In [ ]:
tmdf=tmdf.pivot(index="country", columns="type", values="count").fillna(0)
tmdf.loc[tmdf["TV Show"]>tmdf["Movie"]].sort_values(by="TV Show", ascending=False)

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Above table consist of those countries which have higher count of tv shows as compared to movies
- We can see that Japan, South Korea, Taiwan have the biggest difference in number of tv shows compared to movies. 
- Netflix should add more tv shows in above countries

### <a id='toc1_2_7_'></a>[Rating Analyisis](#toc0_)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=df, x="rating", hue="type");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- From above plot we can say that "TV-MA" and "TV-14 rated content is the most popular
- This plot also shows that majority of the users are of age 14 and above

In [ ]:
plt.figure(figsize=(15, 5))
rating_df = df.loc[df['date_added_year'] >= 2014]
sns.boxplot(data=rating_df, x="rating", y="date_added_year", hue="type")
plt.xticks(rotation=90);

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Majority of the content was added during 2017 - 2020
- It looks like NC-17, NR, TV-Y7-FV rated content was stopped after 2019, indicating less popularity among users

### <a id='toc1_2_8_'></a>[Duration Anaylsis](#toc0_)

#### <a id='toc1_2_8_1_'></a>[TV Show Analysis](#toc0_)

In [ ]:
tv_shows_df["duration"].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=tv_shows_df, x="duration")
plt.title("TV Show Number of Seasons")
plt.xlabel("Number of Seasons");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- According to histogram data, we can see that majority of the tv shows have a duration of 1 season.
- This shows that majority of the users like tv shows that end their story in 1 season. 

In [ ]:
temp_df=tv_shows_df.groupby(["date_added_year","duration"])["show_id"].count().reset_index()
temp_df

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(data=temp_df, x="date_added_year", y="duration", size="show_id")
plt.title("Duration of Shows over years");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- We can say that as small amount of longer duration shows were added to the platform over the years.
- These are the classic shows that have released during the 2000s or before.This indicates that there are some users who would want to watch older shows.
- Netflix should add more of these to the platform.

In [ ]:
temp_df=tv_shows_df.groupby(["date_added_month_name","duration"])["show_id"].count().reset_index()
temp_df

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(data=temp_df, x="date_added_month_name", y="duration", size="show_id");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- There seems to be no pattern of number of season wrt added month of the year

#### <a id='toc1_2_8_2_'></a>[Movie Analysis](#toc0_)

In [ ]:
movies_df["duration"].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(data=movies_df, x="duration", bins=20)
plt.title("Histogram of Movie Durations")
plt.xlabel("Duration (minutes)");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- According to histogram data, we can see that majority of the movies have a duration of 90min - 120min.
- This shows that people are highly interested in movies that are around 2 hrs.

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(data=movies_df, x="date_added_year", y="duration");
plt.title("Duration of Movies");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- There appears to be both increase and decrease in movie duration over the years.

In [ ]:
plt.figure(figsize=(10,5))
sns.scatterplot(data=movies_df, x="date_added_month_name", y="duration");

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- There seems to be no pattern of duration of movies wrt added month of the year

## <a id='toc1_3_'></a>[Recommendations](#toc0_)

### <a id='toc1_3_1_'></a>[General Recommendations](#toc0_)
Netflix is very popular in US with over 37% of content belonging to it. Because of this, it is important to focus on other countries having lesser content.  
By analyzing Netflix data, we have come up with following recommendations:
- Content should be released on platform during the month of July, December and January.
- Content should not be released during the month of February.
- More movies should be produced/added to Netflix as movies are more popular than TV shows. 
- Netflix should added more content in International movies, Dramas and Comedies genres.
- Movie timings should be around 120 minutes.
- TV shows should contain 1 season as these are more popular than shows having multiple season. 
- In Japan, South Korea and Taiwan, TV shows are more popular than movies. More TV shows should be produced in these countries.
- Movies should be added to Netflix within 1 year of release date.
- Netflix should add more of classic tv shows to their platform for catering to older audiences.


### <a id='toc1_3_2_'></a>[Content Recommendations](#toc0_)

In this analysis, we are going to look at improving business in countries that are in top 25 list excluding US. We will do this by ranking Genre, Cast, Rating and Director for each country.  
Netflix can use these ranking to determine the best content for each country.

In [ ]:
filter_country_df = country_df.loc[country_df["country"].isin(improvement_countries)]

#### <a id='toc1_3_2_1_'></a>[Top genres for each country](#toc0_)

In [ ]:
merge_df = df.merge(filter_country_df, on="show_id")
merge_df = merge_df.merge(listed_df, on="show_id")

In [ ]:
gp=merge_df.groupby(["country","type","listed_in"])["show_id"].count().reset_index().rename(columns={"show_id":"count"})
gp["rank"]=gp.groupby(["country","type"])["count"].rank(method="first",ascending=False)
gp=gp.loc[gp["rank"] < 3].sort_values(by='country')
suggested_movie_genres =  gp.loc[(gp["type"]=='Movie')].sort_values(by=["country","rank"]).drop(["type","count"], axis=1)
suggested_tv_genres =  gp.loc[(gp["type"]=='TV Show')].sort_values(by=["country","rank"]).drop(["type","count"], axis=1)

In [ ]:
pd.set_option('display.max_rows', None)
suggested_movie_genres.merge(suggested_tv_genres, on=['country','rank']).rename(columns={'listed_in_x':'Movie Genre', 'listed_in_y':'TV Genres'})

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Above table consist of top movie genre and tv genres in each country.
- Netflix can use this data to determine the most popular genre in each country.  
for eg: Australians like **Drama** movies and **International TV Shows** 

#### <a id='toc1_3_2_2_'></a>[Top rated content for each country](#toc0_)

In [ ]:
gp=merge_df.groupby(["country","type","rating"])["show_id"].count().reset_index().rename(columns={"show_id":"count"})
gp["rank"]=gp.groupby(["country","type"])["count"].rank(method="first",ascending=False)
gp=gp.loc[gp["rank"] < 3].sort_values(by='country')
suggested_tv_ratings =  gp.loc[(gp["type"]=='TV Show')].sort_values(by=["country","rank"]).drop(["count","type"], axis=1)
suggested_movie_ratings =  gp.loc[(gp["type"]=='Movie')].sort_values(by=["country","rank"]).drop(["count","type"], axis=1)

In [ ]:
suggested_movie_ratings.merge(suggested_tv_ratings, on=['country','rank']).rename(columns={'rating_x':'Movie Rating', 'rating_y':'TV Rating'})

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Above table shows top movie and tv ratings in each country.
- Netflix can use this data to predict what type of content users want to watch. 
- For example, Japan likes TV-M movies and TV-14 rated TV shows. This shows that most of the tv users here under the age of 14.

#### <a id='toc1_3_2_3_'></a>[Top cast in each country](#toc0_)

In [ ]:
pd.reset_option('display.max_rows')

In [ ]:
merge_df = df.merge(filter_country_df, on="show_id")
merge_df = merge_df.merge(cast_df, on="show_id")
merge_df=merge_df.loc[merge_df["cast"] != "Unknown"]
gp=merge_df.groupby(["country","type","cast"])["show_id"].count().reset_index().rename(columns={"show_id":"count"})
gp["rank"]=gp.groupby(["country","type"])["count"].rank(method="first",ascending=False)
gp=gp.loc[gp["rank"] < 3].sort_values(by='country')
suggested_movie_cast =  gp.loc[(gp["type"]=='Movie')].sort_values(by=["country","rank"]).drop(["type","count"], axis=1)
suggested_tv_cast =  gp.loc[(gp["type"]=='TV Show')].sort_values(by=["country","rank"]).drop(["count","type"], axis=1)

In [ ]:
pd.set_option('display.max_rows', None)
suggested_movie_cast.merge(suggested_tv_cast, on=['country','rank']).rename(columns={'cast_x':'Movie Cast', 'cast_y':'TV Cast'})

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Above table shows which cast is the most popular in each country.
- Netflix can use this data to predict what type of content users want to watch.
- Using these casts, Netflix can produce their original content and distribute in their respective countries.

#### <a id='toc1_3_2_4_'></a>[Top directors in each country](#toc0_)

In [ ]:
merge_df = df.merge(filter_country_df, on="show_id")
merge_df = merge_df.merge(director_df, on="show_id")
merge_df=merge_df.loc[merge_df["director"] != "Unknown"]
gp=merge_df.groupby(["country","type","director"])["show_id"].count().reset_index().rename(columns={"show_id":"count"})
gp=gp.loc[gp["count"] > 0]
gp["rank"]=gp.groupby(["country","type"])["count"].rank(method="first",ascending=False)
gp=gp.loc[gp["rank"] < 3].sort_values(by='country')
suggested_movie_director =  gp.loc[(gp["type"]=='Movie')].sort_values(by=["country","rank"]).drop(["type","count"], axis=1)
suggested_tv_director =  gp.loc[(gp["type"]=='TV Show')].sort_values(by=["country","rank"]).drop(["count","type"], axis=1)

In [ ]:
pd.set_option('display.max_rows', None)
suggested_movie_director.merge(suggested_tv_director, on=['country','rank'], how="left")\
.rename(columns={'director_x':'Movie Director', 'director_y':'TV Director'})\
.replace(np.nan, '', regex=True)

<b style="padding: 2px 8px;border-radius: 5px;background: #9C27B0;color: #fff;display: inline-block;">Insights</b>

- Above table shows which director is the most popular in each country.
- Netflix can use this data to predict what type of content users want to watch.
- Using these directors, Netflix can produce their original content and distribute in their respective countries.